# Recurrent Neural Network Language Model

This is the "working notebook", with skeleton code to load and train your model, as well as run unit tests. See [rnnlm-instructions.ipynb](rnnlm-instructions.ipynb) for the main writeup.

Run the cell below to import packages.

In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz

# Your code
import rnnlm; reload(rnnlm)
import rnnlm_test; reload(rnnlm_test)

/home/danlee85/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<module 'rnnlm_test' from '/home/danlee85/w266/assignment/a3/lstm/rnnlm_test.py'>

## (a) RNNLM Inputs and Parameters  

### Questions for Part (a)
You should use big-O notation when appropriate (i.e. computing $\exp(\mathbf{v})$ for a vector $\mathbf{v}$ of length $n$ is $O(n)$ operations).  Assume for problems a(1-5) that:   
> -- Cell is one layer,  
> -- the embedding feature length and hidden-layer feature lengths are both H, and   
> -- ignore at the moment batch and max_time dimensions.  

1. Let $\text{CellFunc}$ be a simple RNN cell (see async Section 5.8). Write the cell equation in terms of nonlinearities and matrix multiplication. How many parameters (matrix or vector elements) are there for this cell, in terms of `V` and `H`?
<p>
2. How many parameters are in the embedding layer? In the output layer? (By parameters, we mean total number of matrix elements across all train-able tensors. A $m \times n$ matrix has $mn$ elements.)
<p>
3. How many calculations (floating point operations) are required to compute $\hat{P}(w^{(i+1)})$ for a given *single* target word $w^{(i+1)}$, assuming $w^{(i)}$ given and $h^{(i-1)}$ already computed? How about for *all* target words?
<p>
4. How does your answer to 3. change if we approximate $\hat{P}(w^{(i+1)})$ with a sampled softmax with $k$ samples? How about if we use a hierarchical softmax? (*Recall that hierarchical softmax makes a series of left/right decisions using a binary classifier $P_s(\text{right}) = \sigma(u_s \cdot o^{(i)} + b_s)$ at each split $s$ in the tree.*)
<p>
5. If you have an LSTM with $H = 200$ and use sampled softmax with $k = 100$, what part of the network takes up the most computation time during training? (*Choose "embedding layer", "recurrent layer", or "output layer"*.)

Note: for $A \in \mathbb{R}^{m \times n}$ and $B \in \mathbb{R}^{n \times l}$, computing the matrix product $AB$ takes $O(mnl)$ time.

## (b) Implementing the RNNLM

### Aside: Shapes Review

Before we start, let's review our understanding of the shapes involved in this assignment and how they manifest themselves in the TF API.

As in the [instructions](rnnlm-instructions.ipynb) notebook, $w$ is a matrix of wordids with shape batch_size x max_time.  Passing this through the embedding layer, we retrieve the word embedding for each, resulting in $x$ having shape batch_size x max_time x embedding_dim.  I find it useful to draw this out on a piece of paper.  When you do, you should end up with a rectangular prism with batch_size height, max_time width and embedding_dim depth.  Many tensors in this assignment share this shape (e.g. $o$, the output from the LSTM, which represents the hidden layer going into the softmax to make a prediction at every time step in every batch).

![Three Dimensional Shape](common_shape.png)

Since batch size and sentence length are only resolved when we run the graph, we construct the placeholder using "None" in the dimensions we don't know.  The .shape property renders these as ?s.  This should be familiar to you from batch size handling in earlier assignments, only now there are two dimensions of variable length.

See the next cell for a concrete example (though in practice, we'd use a TensorFlow variable that we can train for the embeddings rather than a static array).  Notice how the shape of x_val matches the shape described earlier in this cell.

In [2]:
tf.reset_default_graph()

wordid_ph = tf.placeholder(tf.int32, shape=[None, None])
embedding_matrix = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]])
x = tf.nn.embedding_lookup(embedding_matrix, wordid_ph)

print('wordid placeholder shape:', wordid_ph.shape)
print('x shape:', x.shape)

sess = tf.Session()
# Two sentences, each with four words.
wordids = [[1, 2, 1, 2], [0, 0, 0, 0]]
x_val = sess.run(x, feed_dict={wordid_ph: wordids})
print('Embeddings shape:', x_val.shape)  # 2 sentences, 4 words, 
print('Embeddings value:\n', x_val)

wordid placeholder shape: (?, ?)
x shape: (?, ?, 3)
Embeddings shape: (2, 4, 3)
Embeddings value:
 [[[2 2 2]
  [3 3 3]
  [2 2 2]
  [3 3 3]]

 [[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]]


### Implmenting the RNNLM

In order to better manage the model parameters, we'll implement our RNNLM in the `RNNLM` class in `rnnlm.py`. We've given you a skeleton of starter code for this, but the bulk of the implementation is left to you.

In [4]:
reload(rnnlm)

TF_GRAPHDIR = "/tmp/w266/a3_graph"

# Clear old log directory.
shutil.rmtree(TF_GRAPHDIR, ignore_errors=True)

lm = rnnlm.RNNLM(V=10000, H=200, num_layers=2)
lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildSamplerGraph()

summary_writer = tf.summary.FileWriter(TF_GRAPHDIR, lm.graph)

The code above will load your implementation, construct the graph, and write a logdir for TensorBoard. You can bring up TensorBoard with:
```
cd assignment/a3
tensorboard --logdir /tmp/w266/a3_graph --port 6006
```
As usual, check http://localhost:6006/ and visit the "Graphs" tab to inspect your implementation. Remember, judicious use of `tf.name_scope()` and/or `tf.variable_scope()` will greatly improve the visualization, and make code easier to debug.

We've provided a few unit tests below to verify some *very* basic properties of your model.

In [5]:
reload(rnnlm); reload(rnnlm_test)
utils.run_tests(rnnlm_test, ["TestRNNLMCore", "TestRNNLMTrain", "TestRNNLMSampler"])

test_shapes_embed (rnnlm_test.TestRNNLMCore) ... ok
test_shapes_output (rnnlm_test.TestRNNLMCore) ... ok
test_shapes_recurrent (rnnlm_test.TestRNNLMCore) ... ok
test_shapes_train (rnnlm_test.TestRNNLMTrain) ... ok
test_shapes_sample (rnnlm_test.TestRNNLMSampler) ... ok

----------------------------------------------------------------------
Ran 5 tests in 2.084s

OK


Note that the error messages are intentionally somewhat spare, and that passing tests are no guarantee of model correctness! Your best chance of success is through careful coding and understanding of how the model works.

## (c) Training your RNNLM (5 points)

We'll give you data loader functions in **`utils.py`**. They work similarly to the loaders in the Week 5 notebook.

Particularly, `utils.rnnlm_batch_generator` will return an iterator that yields minibatches in the correct format. Batches will be of size `[batch_size, max_time]`, and consecutive batches will line up along rows so that the final state $h^{\text{final}}$ of one batch can be used as the initial state $h^{\text{init}}$ for the next.

For example, using a toy corpus:  
*(Ignore the ugly formatter code.)*

In [6]:
toy_corpus = "<s> Mary had a little lamb . <s> The lamb was white as snow . <s>"
toy_corpus = np.array(toy_corpus.split())

html = "<h3>Input words w:</h3>"
html += "<table><tr><th>Batch 0</th><th>Batch 1</th></tr><tr>"
bi = utils.rnnlm_batch_generator(toy_corpus, batch_size=2, max_time=4)
for i, (w,y) in enumerate(bi):
    cols = ["w_%d" % d for d in range(w.shape[1])]
    html += "<td>{:s}</td>".format(utils.render_matrix(w, cols=cols, dtype=object))
html += "</tr></table>"
display(HTML(html))

html = "<h3>Target words y:</h3>"
html += "<table><tr><th>Batch 0</th><th>Batch 1</th></tr><tr>"
bi = utils.rnnlm_batch_generator(toy_corpus, batch_size=2, max_time=4)
for i, (w,y) in enumerate(bi):
    cols = ["y_%d" % d for d in range(y.shape[1])]
    html += "<td>{:s}</td>".format(utils.render_matrix(y, cols=cols, dtype=object))
html += "</tr></table>"
display(HTML(html))

Note that the data we feed to our model will be word indices, but the shape will be the same.

### 1. Implement the `run_epoch` function
We've given you some starter code for logging progress; fill this in with actual call(s) to `session.run` with the appropriate arguments to run a training step. 

Be sure to handle the initial state properly at the beginning of an epoch, and remember to carry over the final state from each batch and use it as the initial state for the next.

**Note:** we provide a `train=True` flag to enable train mode. If `train=False`, this function can also be used for scoring the dataset - see `score_dataset()` below.

#### Questions

1.  Explain what this function does.  Be sure to include the role of `batch_iterator` and what's going on with `h` in the inner loop.

In [7]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=None):
    assert(learning_rate is not None)
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation

    for i, (w, y) in enumerate(batch_iterator):
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})

        feed_dict = {
            lm.input_w_: w,
            lm.target_y_: y,
            lm.initial_h_: h,
            lm.learning_rate_: learning_rate,
            lm.use_dropout_: use_dropout
        }
        ops = [loss, lm.final_h_, train_op]        
        #### YOUR CODE HERE ####
        # session.run(...) the ops with the feed_dict constructed above.
        # Ensure "cost" becomes the value of "loss".
        # Hint: see "ops" for other variables that need updating in this loop.
        

        cost, h, none = session.run(ops, feed_dict = feed_dict)
        
        
    
        #### END(YOUR CODE) ####
        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time

        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print("[batch {:d}]: seen {:d} words at {:.1f} wps, loss = {:.3f}".format(
                i, total_words, avg_wps, avg_cost))
            tick_time = time.time()  # reset time ticker

    return total_cost / total_batches

In [8]:
def score_dataset(lm, session, ids, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils.rnnlm_batch_generator(ids, batch_size=100, max_time=100)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=0.0, train=False, 
                     verbose=False, tick_s=3600)
    print("{:s}: avg. loss: {:.03f}  (perplexity: {:.02f})".format(name, cost, np.exp(cost)))
    return cost

You can use the cell below to verify your implementation of `run_epoch`, and to test your RNN on a (very simple) toy dataset:

In [9]:
reload(rnnlm); reload(rnnlm_test)
th = rnnlm_test.RunEpochTester("test_toy_model")
th.setUp(); th.injectCode(run_epoch, score_dataset)
unittest.TextTestRunner(verbosity=2).run(th)

test_toy_model (rnnlm_test.RunEpochTester) ... 

[batch 0]: seen 50 words at 45.8 wps, loss = 1.849
[batch 201]: seen 10100 words at 4816.3 wps, loss = 0.541
[batch 400]: seen 20050 words at 6469.9 wps, loss = 0.351
[batch 598]: seen 29950 words at 7306.1 wps, loss = 0.274
[batch 799]: seen 40000 words at 7836.9 wps, loss = 0.231
[batch 998]: seen 49950 words at 8182.7 wps, loss = 0.200
[batch 1197]: seen 59900 words at 8428.5 wps, loss = 0.180
[batch 1397]: seen 69900 words at 8619.9 wps, loss = 0.165
[batch 1596]: seen 79850 words at 8761.3 wps, loss = 0.153
[batch 1797]: seen 89900 words at 8885.4 wps, loss = 0.142
[batch 1997]: seen 99900 words at 8983.5 wps, loss = 0.135
[batch 2195]: seen 109800 words at 9057.0 wps, loss = 0.128
[batch 2395]: seen 119800 words at 9126.6 wps, loss = 0.122
[batch 2592]: seen 129650 words at 9176.5 wps, loss = 0.117
[batch 2791]: seen 139600 words at 9225.2 wps, loss = 0.113
[batch 2991]: seen 149600 words at 9273.0 wps, loss = 0.109
[batch 3189]: seen 159500 words at 9308.4 wps, loss = 0.106
[bat

ok

----------------------------------------------------------------------
Ran 1 test in 22.916s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

Note that as above, this is a *very* simple test case that does not guarantee model correctness.

### 2. Run Training

We'll give you the outline of the training procedure, but you'll need to fill in a call to your `run_epoch` function. 

At the end of training, we use a `tf.train.Saver` to save a copy of the model to `/tmp/w266/a3_model/rnnlm_trained`. You'll want to load this from disk to work on later parts of the assignment; see **part (d)** for an example of how this is done.

#### Tuning Hyperparameters
With a sampled softmax loss, the default hyperparameters should train 5 epochs in around 15 minutes on a single-core GCE instance, and reach a training set perplexity between 120-140.

However, it's possible to do significantly better. Try experimenting with multiple RNN layers (`num_layers` > 1) or a larger hidden state - though you may also need to adjust the learning rate and number of epochs for a larger model.

You can also experiment with a larger vocabulary. This will look worse for perplexity, but will be a better model overall as it won't treat so many words as `<unk>`.

#### Notes on Speed

To speed things up, you may want to re-start your GCE instance with more CPUs. Using a 16-core machine will train *very* quickly if using a sampled softmax lost, almost as fast as a GPU. (Because of the sequential nature of the model, GPUs aren't actually much faster than CPUs for training and running RNNs.) The training code will print the words-per-second processed; with the default settings on a single core, you can expect around 8000 WPS, or up to more than 25000 WPS on a fast multi-core machine.

You might also want to modify the code below to only run score_dataset at the very end, after all epochs are completed. This will speed things up significantly, since `score_dataset` uses the full softmax loss - and so often can take longer than a whole training epoch!

#### Submitting your model
You should submit your trained model along with the assignment. Do:
```
cp /tmp/w266/a3_model/rnnlm_trained* .
git add rnnlm_trained*
git commit -m "Adding trained model."
```
Unless you train a very large model, these files should be < 50 MB and no problem for git to handle. If you do also train a large model, please only submit the smaller one.

In [10]:
# Load the dataset
V = 10000
vocab, train_ids, test_ids = utils.load_corpus("brown", split=0.8, V=V, shuffle=42)

[nltk_data] Downloading package brown to /home/danlee85/nltk_data...
[nltk_data]   Package brown is already up-to-date!
Vocabulary: 10,000 types
Loaded 57,340 sentences (1.16119e+06 tokens)
Training set: 45,872 sentences (924,077 tokens)
Test set: 11,468 sentences (237,115 tokens)


In [11]:
# Training parameters
max_time = 25
batch_size = 100
learning_rate = 0.01
num_epochs = 10

# Model parameters
model_params = dict(V=vocab.size, 
                    H=200, 
                    softmax_ns=200,
                    num_layers=2)

TF_SAVEDIR = "/tmp/w266/a3_model"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")

In [12]:
# Will print status every this many seconds
print_interval = 5

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        bi = utils.rnnlm_batch_generator(train_ids, batch_size, max_time)
        print("[epoch {:d}] Starting epoch {:d}".format(epoch, epoch))
        #### YOUR CODE HERE ####
        # Run a training epoch.

        run_epoch(lm, session, bi, train=True, verbose=True,tick_s=10, learning_rate=learning_rate)
        
        
        
        #### END(YOUR CODE) ####
        print("[epoch {:d}] Completed in {:s}".format(epoch, utils.pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
        print("[epoch {:d}]".format(epoch), end=" ")
        score_dataset(lm, session, train_ids, name="Train set")
        print("[epoch {:d}]".format(epoch), end=" ")
        score_dataset(lm, session, test_ids, name="Test set")
        print("")
    
    # Save final model
    saver.save(session, trained_filename)

[epoch 1] Starting epoch 1
[batch 16]: seen 42500 words at 4028.7 wps, loss = 6.765
[batch 33]: seen 85000 words at 4062.5 wps, loss = 6.142
[batch 50]: seen 127500 words at 4080.8 wps, loss = 5.766
[batch 67]: seen 170000 words at 4090.1 wps, loss = 5.533
[batch 84]: seen 212500 words at 4095.3 wps, loss = 5.375
[batch 101]: seen 255000 words at 4095.9 wps, loss = 5.263
[batch 118]: seen 297500 words at 4098.3 wps, loss = 5.169
[batch 135]: seen 340000 words at 4100.5 wps, loss = 5.100
[batch 152]: seen 382500 words at 4100.5 wps, loss = 5.038
[batch 169]: seen 425000 words at 4099.2 wps, loss = 4.985
[batch 186]: seen 467500 words at 4099.4 wps, loss = 4.937
[batch 203]: seen 510000 words at 4100.3 wps, loss = 4.896
[batch 220]: seen 552500 words at 4100.0 wps, loss = 4.856
[batch 237]: seen 595000 words at 4100.3 wps, loss = 4.823
[batch 254]: seen 637500 words at 4101.3 wps, loss = 4.793
[batch 271]: seen 680000 words at 4102.8 wps, loss = 4.765
[batch 288]: seen 722500 words at 41

[batch 271]: seen 680000 words at 4040.0 wps, loss = 3.745
[batch 288]: seen 722500 words at 4039.4 wps, loss = 3.745
[batch 305]: seen 765000 words at 4039.0 wps, loss = 3.744
[batch 322]: seen 807500 words at 4038.4 wps, loss = 3.744
[batch 339]: seen 850000 words at 4038.2 wps, loss = 3.744
[batch 356]: seen 892500 words at 4036.3 wps, loss = 3.744
[batch 373]: seen 935000 words at 4036.2 wps, loss = 3.743
[epoch 6] Completed in 0:04:00
[epoch 6] Train set: avg. loss: 4.925  (perplexity: 137.65)
[epoch 6] Test set: avg. loss: 5.031  (perplexity: 153.06)

[epoch 7] Starting epoch 7
[batch 16]: seen 42500 words at 4052.1 wps, loss = 3.724
[batch 33]: seen 85000 words at 4057.2 wps, loss = 3.720
[batch 50]: seen 127500 words at 4062.1 wps, loss = 3.721
[batch 67]: seen 170000 words at 4070.3 wps, loss = 3.720
[batch 84]: seen 212500 words at 4069.2 wps, loss = 3.718
[batch 101]: seen 255000 words at 4068.5 wps, loss = 3.715
[batch 118]: seen 297500 words at 4068.5 wps, loss = 3.714
[ba

## (d) Sampling Sentences (5 points)

If you didn't already in **part (b)**, implement the `BuildSamplerGraph()` method in `rnnlm.py` See the function docstring for more information.

#### Implement the `sample_step()` method below (5 points)
This should access the Tensors you create in `BuildSamplerGraph()`. Given an input batch and initial states, it should return a vector of shape `[batch_size,1]` containing sampled indices for the next word of each batch sequence.

Run the method using the provided code to generate 10 sentences.

In [13]:
def sample_step(lm, session, input_w, initial_h):
    """Run a single RNN step and return sampled predictions.
  
    Args:
      lm : rnnlm.RNNLM
      session: tf.Session
      input_w : [batch_size] vector of indices
      initial_h : [batch_size, hidden_dims] initial state
    
    Returns:
      final_h : final hidden state, compatible with initial_h
      samples : [batch_size, 1] vector of indices
    """
    # Reshape input to column vector
    input_w = np.array(input_w, dtype=np.int32).reshape([-1,1])
  
    #### YOUR CODE HERE ####
    # Run sample ops

    initial_h = session.run(lm.initial_h_, {lm.input_w_: input_w})
    
    feed_dict = {
        lm.input_w_: input_w,
        lm.initial_h_: initial_h,
        }
    
    ops = [lm.final_h_, lm.pred_samples_]

    final_h, samples = session.run(ops, feed_dict = feed_dict)

    

    #### END(YOUR CODE) ####
    # Note indexing here: 
    #   [batch_size, max_time, 1] -> [batch_size, 1]
    return final_h, samples[:,-1,:]

In [14]:
# Same as above, but as a batch
max_steps = 20
num_samples = 10
random_seed = 42

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildSamplerGraph()

with lm.graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(random_seed)
    
    # Load the trained model
    saver.restore(session, trained_filename)

    # Make initial state for a batch with batch_size = num_samples
    w = np.repeat([[vocab.START_ID]], num_samples, axis=0)
    h = session.run(lm.initial_h_, {lm.input_w_: w})
    # We'll take one step for each sequence on each iteration 
    for i in range(max_steps):
        h, y = sample_step(lm, session, w[:,-1:], h)
        w = np.hstack((w,y))

    # Print generated sentences
    for row in w:
        for i, word_id in enumerate(row):
            print(vocab.id_to_word[word_id], end=" ")
            if (i != 0) and (word_id == vocab.START_ID):
                break
        print("")

INFO:tensorflow:Restoring parameters from /tmp/w266/a3_model/rnnlm_trained
<s> no curve . <s> 
<s> he turned in <unk> agreement , as far to speak over addition to lock on your talent . <s> 
<s> <unk> , nuts <unk> rite of social craft <unk> doses of building , but a <unk> anglo-saxon george nixon of 
<s> and its mother's meat doesn't learned that he entered the <unk> might discover <unk> , however , served off him 
<s> she had a honest treatments . <s> 
<s> he knew it is <unk> cloth virginia . <s> 
<s> see first situation of valley . <s> 
<s> so treatment you'll become like a walnut milling ribs were <unk> and no <unk> as love questions , but <unk> 
<s> flat minutes . <s> 
<s> he nodded DGDGDGDG . <s> 


## (e) Linguistic Properties (5 points)

Now that we've trained our RNNLM, let's test a few properties of the model to see how well it learns linguistic phenomena. We'll do this with a scoring task: given two or more test sentences, our model should score the more plausible (or more correct) sentence with a higher log-probability.

We'll define a scoring function to help us:

In [15]:
def score_seq(lm, session, seq, vocab):
    """Score a sequence of words. Returns total log-probability."""
    padded_ids = vocab.words_to_ids(utils.canonicalize_words(["<s>"] + seq + ["</s>"], 
                                                             wordset=vocab.word_to_id))
    w = np.reshape(padded_ids[:-1], [1,-1])
    y = np.reshape(padded_ids[1:],  [1,-1])
    h = session.run(lm.initial_h_, {lm.input_w_: w})
    feed_dict = {lm.input_w_:w,
                 lm.target_y_:y,
                 lm.initial_h_:h,
                 lm.dropout_keep_prob_: 1.0}
    # Return log(P(seq)) = -1*loss
    return -1*session.run(lm.loss_, feed_dict)

def load_and_score(inputs, sort=False):
    """Load the trained model and score the given words."""
    lm = rnnlm.RNNLM(**model_params)
    lm.BuildCoreGraph()
    
    with lm.graph.as_default():
        saver = tf.train.Saver()

    with tf.Session(graph=lm.graph) as session:  
        # Load the trained model
        saver.restore(session, trained_filename)

        if isinstance(inputs[0], str) or isinstance(inputs[0], bytes):
            inputs = [inputs]

        # Actually run scoring
        results = []
        for words in inputs:
            score = score_seq(lm, session, words, vocab)
            results.append((score, words))

        # Sort if requested
        if sort: results = sorted(results, reverse=True)

        # Print results
        for score, words in results:
            print("\"{:s}\" : {:.02f}".format(" ".join(words), score))

Now we can test as:

In [16]:
sents = ["once upon a time",
         "the quick brown fox jumps over the lazy dog"]
load_and_score([s.split() for s in sents])

INFO:tensorflow:Restoring parameters from /tmp/w266/a3_model/rnnlm_trained
"once upon a time" : -8.26
"the quick brown fox jumps over the lazy dog" : -7.72


### 1. Number agreement

Compare **"the boy and the girl [are/is]"**. Which is more plausible according to your model?

If your model doesn't order them correctly (*this is OK*), why do you think that might be? (answer in cell below)

In [17]:
#### YOUR CODE HERE ####

sents = ["the boy and the girl are",
        "the boy and the girl is"]
load_and_score([s.split() for s in sents])

#### END(YOUR CODE) ####

INFO:tensorflow:Restoring parameters from /tmp/w266/a3_model/rnnlm_trained
"the boy and the girl are" : -5.95
"the boy and the girl is" : -5.79


### 2. Type/semantic agreement

Compare:
- **"peanuts are my favorite kind of [nut/vegetable]"**
- **"when I'm hungry I really prefer to [eat/drink]"**

Of each pair, which is more plausible according to your model?

How would you expect a 3-gram language model to perform at this example? How about a 5-gram model? (answer in cell below)

In [18]:
#### YOUR CODE HERE ####

sents = ["peanuts are my favorite kind of nut",
         "peanuts are my favorite kind of vegetable",
         "when I'm hungry I really prefer to eat",
         "when I'm hungry I really prefer to drink"]
load_and_score([s.split() for s in sents])


#### END(YOUR CODE) ####

INFO:tensorflow:Restoring parameters from /tmp/w266/a3_model/rnnlm_trained
"peanuts are my favorite kind of nut" : -7.27
"peanuts are my favorite kind of vegetable" : -6.99
"when I'm hungry I really prefer to eat" : -7.47
"when I'm hungry I really prefer to drink" : -7.65


### 3. Adjective ordering (just for fun)

Let's repeat the exercise from Week 2:

![Adjective Order](adjective_order.jpg)
*source: https://twitter.com/MattAndersonBBC/status/772002757222002688?lang=en*

We'll consider a toy example (literally), and consider all possible adjective permutations.

Note that this is somewhat sensitive to training, and even a good language model might not get it all correct. Why might the NN fail, if the trigram model from Week 2 was able to solve it?

In [19]:
prefix = "I have lots of".split()
noun = "toys"
adjectives = ["square", "green", "plastic"]
inputs = []
for adjs in itertools.permutations(adjectives):
    words = prefix + list(adjs) + [noun]
    inputs.append(words)
    
load_and_score(inputs, sort=True)

INFO:tensorflow:Restoring parameters from /tmp/w266/a3_model/rnnlm_trained
"I have lots of green plastic square toys" : -8.04
"I have lots of plastic green square toys" : -8.08
"I have lots of plastic square green toys" : -8.13
"I have lots of green square plastic toys" : -8.14
"I have lots of square plastic green toys" : -8.26
"I have lots of square green plastic toys" : -8.29
